In [1]:
import findspark
findspark.init()

from pyspark import *
from pyspark.sql.functions import desc, col, rand
from pyspark.sql import *
from graphframes import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

import os
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import sys
from sympy.ntheory.generate import nextprime
import time
from tqdm import tqdm
import copy
from math import comb
import multiprocessing as mp

from utils import read_data

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# https://graphframes.github.io/graphframes/docs/_site/quick-start.html
# https://stackoverflow.com/questions/65011599/how-to-start-graphframes-on-spark-on-pyspark-on-juypter-on-docker
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell'
os.environ['PYSPARK_PYTHON'] = sys.executable

In [5]:
spark = SparkSession.builder.appName('hw2').master("local[2]").getOrCreate()

In [6]:
data_path = "data/T10I4D100K.dat"

In [9]:
# for multiprocessing
nprocs = mp.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 8


In [11]:
data_rdd, items_rdd, items_counts_rdd = read_data(data_path=data_path, spark=spark)
data_rdd_c = data_rdd.collect()

In [19]:
print("An example basket:")
print(data_rdd.take(1)[0])

n_baskets = data_rdd.count()
print(f"n_baskets = {n_baskets}")

n_items = items_rdd.count()
print(f"n_items = {n_items}")

print("Example item counts:")
print(items_counts_rdd.take(10))

An example basket:
{448, 834, 164, 775, 328, 687, 240, 368, 274, 561, 52, 630, 730, 825, 538, 25}
n_baskets = 100000
n_items = 870
Example item counts:
[(448, 1370), (834, 1373), (164, 744), (328, 663), (240, 1399), (368, 7828), (274, 2628), (52, 1983), (630, 1523), (538, 3982)]


In [21]:
def get_singletons(items_counts_rdd, s):
    """Read data given a path.

    Parameters
    ----------
    items_counts_rdd : pyspark.rdd.PipelinedRDD
        A list of tuples of items and their counts. All items are represented as integers.

    s : int
        Support, min number of occurence across baskets.

    Returns
    -------
    singletons_rdd : pyspark.rdd.PipelinedRDD
        A list of items and their counts/support.
        E.g.:
        [({448}, 1370),
         ({834}, 1373),
         ({240}, 1399),
         ({368}, 7828),
         ({274}, 2628),
         ({52}, 1983),
         ({630}, 1523),
         ({538}, 3982),
         ({704}, 1794),
         ({814}, 1672)]
    """
    singletons_rdd = items_counts_rdd.filter(lambda x: s <= x[1])
    singletons_rdd = singletons_rdd.map(lambda x: (set([x[0]]), x[1]))
    return singletons_rdd

In [22]:
def construct_itemsets(k, itemsets_frequent_rdd, from_ckpt=False):
    """Construct itemsets, or generate candidates for filtering in the Apriori algorithm.

    Parameters
    ----------
    k : int
        Length of proposed itemsets.
    
    itemsets_frequent_rdd : pyspark.rdd.PipelinedRDD
        A list of tuples of itemsets and their counts, from previous step. All items are represented as integers.

    from_ckpt : bool
        If True, load candidate sets from checkpoint, else compute them.

    Returns
    -------
    candidates : pyspark.rdd.PipelinedRDD
        A list of list where the first element is the id of the candidate itemset
        and the second element is the itemset.
        E.g.:
        [[0, {413, 494}],
         [1, {874, 978}],
         [2, {701, 946}],
         [3, {335, 804}],
         [4, {576, 583}],
         [5, {242, 684}],
         [6, {597, 641}],
         [7, {581, 766}],
         [8, {335, 538}],
         [9, {39, 884}],
         [10, {516, 854}],
         [11, {115, 735}],
         [12, {126, 952}],
         [13, {854, 895}],
         [14, {682, 740}],
         [15, {774, 984}],
         [16, {468, 984}],
         [17, {738, 749}],
         [18, {675, 790}],
         [19, {529, 600}]]
    """
    assert 1 < k, f"k={k}, needs to be 1<k"
    
    if from_ckpt:
        candidates = np.load(f'ckpt/candidates_k_{k}.npy', allow_pickle=True)
        candidates = spark.sparkContext.parallelize(candidates.tolist())
        print(f"loaded proposed n={candidates.count()} candidates")
    else:
        # get singelton itemsets
        singletons_rdd = itemsets_frequent_rdd.filter(lambda x: len(x[0]) == 1)
        # get itemsets of length k-1
        k_minus_1_tons_rdd = itemsets_frequent_rdd.filter(lambda x: len(x[0]) == k-1)
        # use singletions an k-1tons to prodcue cartesian product of itemsets
        singletons_rdd = singletons_rdd.map(lambda x: x[0])
        k_minus_1_tons_rdd = k_minus_1_tons_rdd.map(lambda x: x[0])
        cartesian_rdd = singletons_rdd.cartesian(k_minus_1_tons_rdd)
        # keep from cartesian the itemsets that are k long
        cartesian_k_rdd = cartesian.map(lambda x: x[0].union(x[1])).filter(lambda x: len(x) == k)
        cartesian_k = cartesian_k.collect()
        # remove duplicate sets (could have due to cartesian)
        cartesian_k_no_dupl = [set(item) for item in set(frozenset(item) for item in cartesian_k)]
        # make candidate list with structure
        candidates_list = [(x, 0) for idx, x in enumerate(cartesian_k_no_dupl)]
        candidates = spark.sparkContext.parallelize(candidates_list)
        # unpruned lenght
        n_before_prune = candidates.count()
        # save as ckpt
        np.save(f'ckpt/candidates_notpruned_k_{k}', np.array(candidates.collect()))

        # prune candidates: only keep the ones whose all subsets are also frequent itemsets 
        # subsets are of legnth=1,...,k-1
        for i in range(1, k):
            # k choose i, n_comb is the number of possible subsets in  
            n_comb = comb(k,i)
            # get frequent itemsets of lenght i
            itemsets_i = itemsets_frequent_rdd.filter(lambda x: len(x[0]) == i).collect()
            # for each candidate 
            candidates = \
                candidates.map(lambda x: (x[0], sum([len(x[0].intersection(s[0])) == i for s in itemsets_i])))\
                .filter(lambda x: n_comb == x[1])

        candidates = candidates.map(lambda x: x[0]).zipWithIndex().map(lambda x: (x[1], x[0]))
        n_after_prune = candidates.count()

        np.save(f'ckpt/candidates_k_{k}', np.array(candidates.collect()))

        print(f"proposing n={candidates.count()} candidates (n_pruned={n_before_prune - n_after_prune})")
    
    return candidates

def f(candidate, data_rdd_c, k):
        return len(list(filter(lambda x: len(x) == k, map(lambda x: x & candidate, data_rdd_c))))

def filter_itemsets(candidates_rdd, k, s, itemsets_frequent_rdd, from_ckpt=False):
    if from_ckpt:
        start_time = time.time()
        print("Filtering loading from file...")
        res = np.load(f'ckpt/filtered_candidates_k_{k}_s_{s}.npy', allow_pickle=True)
        res = spark.sparkContext.parallelize(res.tolist())
        end_time = time.time()
        print(f"k={k}, t={end_time - start_time}, n={res.count()}")
    else:
        start_time = time.time()
        print("Staring filtering...")

        candidates = candidates_rdd.collect()

        pool = mp.Pool(processes=nprocs)
        supports = pool.starmap(f, [(c,data_rdd_c,k) for (idx, c) in candidates])

        res = \
            spark.sparkContext.parallelize(candidates)\
            .filter(lambda x: s <= supports[x[0]]).map(lambda x: (x[1], supports[x[0]]))

        np.save(f'ckpt/filtered_candidates_k_{k}_s_{s}', np.array(res.collect()))

        end_time = time.time()
        print(f"k={k}, t={end_time - start_time}, n={res.count()}")
    
    return res

In [23]:
s = 1000
from_ckpt = True

itemsets_frequent_rdd_1 = get_singletons(items_counts_rdd=items_counts_rdd, s=s)
itemsets_frequent_rdd_1.count()

375

In [24]:
itemsets_frequent_rdd_1.take(10)

[({448}, 1370),
 ({834}, 1373),
 ({240}, 1399),
 ({368}, 7828),
 ({274}, 2628),
 ({52}, 1983),
 ({630}, 1523),
 ({538}, 3982),
 ({704}, 1794),
 ({814}, 1672)]

In [25]:
candidates_2 = construct_itemsets(k=2, itemsets_frequent_rdd=itemsets_frequent_rdd_1, from_ckpt=from_ckpt)
candidates_2.take(20)

loaded proposed n=70125 candidates


[[0, {413, 494}],
 [1, {874, 978}],
 [2, {701, 946}],
 [3, {335, 804}],
 [4, {576, 583}],
 [5, {242, 684}],
 [6, {597, 641}],
 [7, {581, 766}],
 [8, {335, 538}],
 [9, {39, 884}],
 [10, {516, 854}],
 [11, {115, 735}],
 [12, {126, 952}],
 [13, {854, 895}],
 [14, {682, 740}],
 [15, {774, 984}],
 [16, {468, 984}],
 [17, {738, 749}],
 [18, {675, 790}],
 [19, {529, 600}]]

In [26]:
new_itemsets_frequent_rdd_2 = \
    filter_itemsets(candidates_rdd=candidates_2, k=2, s=s, itemsets_frequent_rdd=itemsets_frequent_rdd_1, from_ckpt=from_ckpt)

itemsets_frequent_rdd_2 = itemsets_frequent_rdd_1.union(new_itemsets_frequent_rdd_2)
print(itemsets_frequent_rdd_2.count())

Filtering loading from file...
k=2, t=0.004158735275268555, n=9
384


In [29]:
itemsets_frequent_rdd_2.take(10)

[({448}, 1370),
 ({834}, 1373),
 ({240}, 1399),
 ({368}, 7828),
 ({274}, 2628),
 ({52}, 1983),
 ({630}, 1523),
 ({538}, 3982),
 ({704}, 1794),
 ({814}, 1672)]

In [30]:
candidates_3 = construct_itemsets(k=3, itemsets_frequent_rdd=itemsets_frequent_rdd_2, from_ckpt=from_ckpt)
candidates_3.take(20)

loaded proposed n=1 candidates


[[0, {39, 704, 825}]]

In [31]:
new_itemsets_frequent_rdd_3 = \
    filter_itemsets(candidates_rdd=candidates_3, k=3, s=s, itemsets_frequent_rdd=itemsets_frequent_rdd_2, from_ckpt=from_ckpt)

itemsets_frequent_rdd_3 = itemsets_frequent_rdd_2.union(new_itemsets_frequent_rdd_3)
print(itemsets_frequent_rdd_3.count())

Filtering loading from file...
k=3, t=0.0035021305084228516, n=1
385


In [32]:
candidates_4 = construct_itemsets(k=4, itemsets_frequent_rdd=itemsets_frequent_rdd_3, from_ckpt=False)
candidates_4.take(20)

proposing n=0 candidates (n_pruned=372)


[]

In [33]:
itemsets_frequent_rdd_3.count()

385

In [34]:
itemsets_frequent_rdd_3.take(500)

[({448}, 1370),
 ({834}, 1373),
 ({240}, 1399),
 ({368}, 7828),
 ({274}, 2628),
 ({52}, 1983),
 ({630}, 1523),
 ({538}, 3982),
 ({704}, 1794),
 ({814}, 1672),
 ({120}, 4973),
 ({674}, 2527),
 ({854}, 2847),
 ({950}, 1463),
 ({964}, 1518),
 ({422}, 1255),
 ({738}, 2129),
 ({708}, 1090),
 ({294}, 1445),
 ({966}, 3921),
 ({978}, 1141),
 ({766}, 6265),
 ({104}, 1158),
 ({620}, 2100),
 ({798}, 3103),
 ({682}, 4132),
 ({970}, 2086),
 ({782}, 2767),
 ({658}, 1881),
 ({214}, 1893),
 ({350}, 3069),
 ({390}, 2685),
 ({530}, 1263),
 ({914}, 4037),
 ({280}, 2108),
 ({932}, 1786),
 ({192}, 2004),
 ({208}, 1483),
 ({720}, 3864),
 ({618}, 1337),
 ({496}, 1428),
 ({706}, 1923),
 ({878}, 2047),
 ({276}, 2479),
 ({960}, 2732),
 ({424}, 1448),
 ({490}, 1066),
 ({910}, 1695),
 ({130}, 1711),
 ({392}, 2420),
 ({862}, 3649),
 ({900}, 1165),
 ({78}, 2471),
 ({778}, 2514),
 ({572}, 1589),
 ({290}, 1793),
 ({614}, 3134),
 ({266}, 1022),
 ({458}, 1124),
 ({944}, 2794),
 ({888}, 3686),
 ({480}, 2309),
 ({70}, 24

In [35]:
itemsets_frequent = itemsets_frequent_rdd_3.collect()
itemsets_frequent_notsingle_rdd = itemsets_frequent_rdd_3.filter(lambda x: 1 < len(x[0]))
itemsets_frequent_notsingle = itemsets_frequent_notsingle_rdd.collect()
itemsets_frequent_notsingle

[[{368, 829}, 1194],
 [{390, 722}, 1042],
 [{789, 829}, 1194],
 [{704, 825}, 1102],
 [{39, 704}, 1107],
 [{227, 390}, 1049],
 [{368, 682}, 1193],
 [{217, 346}, 1336],
 [{39, 825}, 1187],
 [{39, 704, 825}, 1035]]

In [ ]:
from itertools import chain, combinations

In [ ]:
def get_subsets(myset):
    max_ = len(myset)
    min_ = 0
    # https://stackoverflow.com/questions/1482308/how-to-get-all-subsets-of-a-set-powerset
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    subsets = \
        [set(subset) for subset in list(chain.from_iterable(combinations(myset, r) for r in range(len(rule_set)+1)))]
    valid_subsets = [subset for subset in subsets if 0 < len(subset) < max_]
    return valid_subsets

In [ ]:
myset_support = itemsets_frequent_notsingle[0]
print(myset_support)
myset = myset_support[0]
subsets = get_subsets(myset)
subsets

In [ ]:
myset

In [ ]:
rule_sets = [(subset, myset.difference(subset)) for subset in subsets]
rule_sets

In [ ]:
def get_conf_rule_set(rule_set, itemsets_frequent):
    i = rule_set[0]
    j = rule_set[1]
    union = i.union(j)
    
    union_idx = \
        np.where(np.array(union) == np.array([itemset_frequent[0] for itemset_frequent in itemsets_frequent]))[0][0]
    i_idx = \
        np.where(np.array(i) == np.array([itemset_frequent[0] for itemset_frequent in itemsets_frequent]))[0][0]
    
    union_support = itemsets_frequent[union_idx][1]
    print(union_support)
    i_support = itemsets_frequent[i_idx][1]
    
    conf = union_support / i_support
    
    return conf
    
rule_set = rule_sets[0]
print(rule_set)
get_conf_rule_set(rule_set, itemsets_frequent)

In [ ]:
np.where(np.array({3,2}) == np.array([{2,1}, {3,4}, {2,3}]))

In [ ]:
def get_association_rules(itemsets_frequent, c_thresh=None):
    itemsets_frequent_not_singleton = list(filter(lambda x: 1 < len(x[0]), itemsets_frequent))
    print(itemsets_frequent_not_singleton)
    
    association_rules = []
    
    for itemset_frequent, union_support in itemsets_frequent_not_singleton:
        print(itemset_frequent)
        print(union_support)
        
        subsets = get_subsets(itemset_frequent)
        
        rule_sets = [(subset, itemset_frequent.difference(subset)) for subset in subsets]
        
        association_rule_sets = []
        
        for rule_set in rule_sets:
            i = rule_set[0]
            i_idx = \
                np.where(np.array(i) == np.array([itemset_frequent[0] for itemset_frequent in itemsets_frequent]))[0][0]
            i_support = itemsets_frequent[i_idx][1]
            
            conf = union_support / i_support
            
            if c_thresh <= conf:
                association_rule_sets.append((rule_set, conf))
            
        association_rules.append((itemset_frequent, association_rule_sets))
        
    return association_rules

In [ ]:
c_thresh = 0.5
association_rules = get_association_rules(itemsets_frequent=itemsets_frequent, c_thresh=c_thresh)
print(association_rules)

In [ ]:
def pretty_print_association_rules(association_rules):
    for frequent_itemset, association_rule_sets in association_rules:
        print(f"frequent itemset: {frequent_itemset}")
        
        for association_rule_set, confidence in association_rule_sets:
            print(f"\tassociation rule: {association_rule_set[0]} -> {association_rule_set[1]}"
                  f" with confidence={confidence:.4f}")
        print()
            
pretty_print_association_rules(association_rules)